In [3]:
import requests 
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
import chromedriver_binary
from webdriver_manager.chrome import ChromeDriverManager
import time

In [4]:
link = 'https://stock-pictures.netlify.app'

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [02:21<00:00, 50.2kB/s]
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_17452\2767233116.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [6]:
browser = driver
browser.get(link)

In [7]:
soup = BeautifulSoup(browser.page_source, 'html.parser')
soup.text

# Loading the data of all the contents can be done in two ways:
# One is by manually scrolling to the bottom then loading the page.


'Stock ImagesYou need to enable JavaScript to run this app.Stock Images with TagsLoading...Scrapped and Made with ❤ '

### Enabling the auto scroll method

In [8]:
# auto scroll using the browser
browser.execute_script('scrollTo(0,100)')   # arguments of scroll to are in pixels
time.sleep(1)


### Scrolling to reach the bottom

In [9]:
for i in tqdm(range(0,2500000,1000)):
    browser.execute_script('scrollTo(0,' + str(i) + ')') 
    time.sleep(0.3)

100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [14:09<00:00,  2.94it/s]


### Scraping the page source

In [10]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [11]:
data = []

for sp in tqdm(soup.find_all('div', class_ = 'container')):
    img_link = sp.find('img').get('src')

    tags = ', '.join(sp.find('span', class_ = 'tag-color').text[7:].strip().replace(' ','').split(','))
    
    likes = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.split(' ')[0])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.split(' ')[0])
    
    data.append([img_link, tags, likes, comments])
    

100%|████████████████████████████████████████████████████████████████████████████| 9104/9104 [00:01<00:00, 5228.50it/s]


In [12]:
# len(soup.find_all('div', class_ = 'container'))

### Converting into dataframe

In [13]:
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])

df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, BlueSky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"CherryBlossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"CapeMarguerite, Flower, Plant",39,15


In [14]:
df.isnull().sum()

img_link    0
tags        0
likes       0
comments    0
dtype: int64

### Checking the duplicate values

In [15]:
# There are some duplicate images in the dataframe
df['img_link'].describe()

count                                                  9104
unique                                                 9088
top       https://cdn.pixabay.com/photo/2022/03/06/05/30...
freq                                                      2
Name: img_link, dtype: object

In [16]:
# df['img_link'].drop_duplicates()

0       https://cdn.pixabay.com/photo/2022/03/06/05/30...
1       https://cdn.pixabay.com/photo/2022/04/07/11/45...
2       https://cdn.pixabay.com/photo/2022/02/28/15/28...
3       https://cdn.pixabay.com/photo/2022/04/04/02/52...
4       https://cdn.pixabay.com/photo/2022/04/09/18/06...
                              ...                        
9099    https://cdn.pixabay.com/photo/2018/01/08/18/51...
9100    https://cdn.pixabay.com/photo/2021/09/22/01/42...
9101    https://cdn.pixabay.com/photo/2022/01/09/15/58...
9102    https://cdn.pixabay.com/photo/2021/08/10/11/59...
9103    https://cdn.pixabay.com/photo/2021/10/03/01/01...
Name: img_link, Length: 9088, dtype: object

In [17]:
# Removing the duplicates 
df = df.drop_duplicates()

### Converting into the csv file

In [19]:
df.to_csv('infinite-scroll-data.csv', index = False)